<a href="https://colab.research.google.com/github/StrayBeing/AI_Lab/blob/main/Lab490%25Test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [95]:
import numpy as np
import tensorflow as tf
from sklearn.utils import class_weight
from tensorflow.keras.layers import LeakyReLU

In [96]:
npz = np.load('Audiobooks_data_train.npz')
train_inputs = npz['inputs'].astype(np.float32)
train_targets = npz['targets'].astype(np.int32)
npz = np.load('Audiobooks_data_validation.npz')
validation_inputs, validation_targets = npz['inputs'].astype(np.float32), npz['targets'].astype(np.int32)
npz = np.load('Audiobooks_data_test.npz')
test_inputs, test_targets = npz['inputs'].astype(np.float32), npz['targets'].astype(np.int32)


In [97]:
input_size = 10
output_size = 2
hidden_layer_size = 100
model = tf.keras.Sequential([
    tf.keras.layers.Dense(512),
    LeakyReLU(alpha=0.1),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.4),

    tf.keras.layers.Dense(256),
    LeakyReLU(alpha=0.1),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.3),

    tf.keras.layers.Dense(128),
    LeakyReLU(alpha=0.1),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.2),

    tf.keras.layers.Dense(64),
    LeakyReLU(alpha=0.1),
    tf.keras.layers.Dense(2, activation='softmax')
])

model.compile(
    optimizer=tf.keras.optimizers.Nadam(learning_rate=0.001),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

/usr/local/lib/python3.11/dist-packages/keras/src/layers/activations/leaky_relu.py:41: UserWarning: Argument `alpha` is deprecated. Use `negative_slope` instead.
  warnings.warn(


In [98]:
batch_size = 100
max_epochs = 100
early_stopping = tf.keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)
class_weights = class_weight.compute_class_weight(
    class_weight='balanced',
    classes=np.unique(train_targets),
    y=train_targets
)

class_weights = dict(enumerate(class_weights))

model.fit(train_inputs,
         train_targets,
         batch_size=batch_size,
         epochs=max_epochs,
         callbacks=[early_stopping],
         validation_data=(validation_inputs, validation_targets),
          class_weight=class_weights,
         verbose=1
          )

Epoch 1/100
36/36 ━━━━━━━━━━━━━━━━━━━━ 7s 30ms/step - accuracy: 0.6256 - loss: 0.7869 - val_accuracy: 0.7002 - val_loss: 0.5448
Epoch 2/100
36/36 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - accuracy: 0.7399 - loss: 0.4807 - val_accuracy: 0.7718 - val_loss: 0.5131
Epoch 3/100
36/36 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - accuracy: 0.7410 - loss: 0.4713 - val_accuracy: 0.6711 - val_loss: 0.5214
Epoch 4/100
36/36 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - accuracy: 0.7583 - loss: 0.4473 - val_accuracy: 0.7204 - val_loss: 0.4803
Epoch 5/100
36/36 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - accuracy: 0.7466 - loss: 0.4348 - val_accuracy: 0.7830 - val_loss: 0.4421
Epoch 6/100
36/36 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - accuracy: 0.7657 - loss: 0.4167 - val_accuracy: 0.7539 - val_loss: 0.4410
Epoch 7/100
36/36 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - accuracy: 0.7868 - loss: 0.4009 - val_accuracy: 0.7293 - val_loss: 0.4298
Epoch 8/100
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.7804 - loss: 0.3967 - val_accuracy: 0.

In [92]:
test_loss, test_accuracy = model.evaluate(test_inputs, test_targets)
print('\nTest loss: {0:.2f}. Test accuracy: {1:.2f}%'.format(test_loss, test_accuracy*100.))


14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8031 - loss: 0.3907 

Test loss: 0.38. Test accuracy: 81.03%
